In [1]:
class Literal:
    """ 
    Class representing a logical literal
    Attributes: 
    name: string containing name of the literal 
    sign: boolean value indicating the sign of the literal
    Methods:
    __repr__: print out the literal with correct sign
    __neg__: negate a literal
    __eq__: compare two literals by their names 
    __hash__: return hash value of a literal name
    
    """
    def __init__(self,name='', sign = True):
        self.name = name 
        self.sign = sign 
    
    def __repr__(self): 
        if self.sign:
            return self.name
        elif not self.sign:
            return('-' + self.name)
        
    def __neg__(self):
        return Literal(self.name, not self.sign)
    
    def __eq__(self, other): 
        return self.name == other.name
    
    def __hash__(self):
        # Define the hash value to put in a set 
        return hash(self.name)

In [2]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

In [3]:
def is_true(clause, model = {}):
    """ Evaluate if a clause is true given
    a model. If the truth value of a clause 
    is undefined, return None
    """
    truth_values = []
    for s in clause:
        if s in model:
            if s.sign == True:
                truth_values.append(model[s])
            else:
                truth_values.append(not model[s])
    #The whole clause is True as long as one of the unit is True 
    #because they are related by disjunction
    if True in truth_values: 
        return True
    #The whole clause is False if no unit evaluates to True 
    elif set(truth_values) == {False} and len(truth_values) == len(clause):
        return False 

In [4]:
is_true({A,B}, {A: False, B: True})

True

In [5]:
is_true({A,B}, {A: False, B: False})

False

In [6]:
print (is_true({A,B}, {}))

None


In [7]:
def most_common(symbols,clauses):
    """ Find the most common literal in the KB 
    and return that literal together with the list of the 
    remaining literals. 
    """
    #Turn the list of dictionary into a 1-d list:
    flat = [s for c in clauses for s in c if s in symbols]
    #Find the most common literal: 
    common = max(set(flat), key=flat.count)
    
    #Put the remaining literals into a new list. 
    remaining = [s for s in symbols if s != common]
    return(common, remaining)

In [8]:
most_common([A,B,C],[{B},{B,C}])

(B, [A, C])

In [9]:
def find_pure_symbols(symbols, clauses):
    """ Find all the pure symbols given a set of clauses.
    Return a dictionary, the value of symbol indicates its sign. 
    Symbol with value of None is not pure. 
    """
    flat = [s for c in clauses for s in c if s in symbols]
    literals = {}
    for i in flat:
        if i not in literals:
            literals[i] = i.sign 
        else:
            if i.sign != literals[i]:
                literals[i] = None
    return literals

In [10]:
find_pure_symbols([A,B,C], [{A},{B,-A},{C},{-C,-B}])

{A: None, B: None, C: None}

In [11]:
find_pure_symbols([A,B,C], [{A},{-B,A},{C},{-C,-B}])

{-B: False, A: True, C: None}

In [12]:
def one_pure_symbol(symbols, clauses):
    """ Return only one of the pure symbols found (in its positive form) 
    and its real sign. 
    """
    l = find_pure_symbols(symbols, clauses)
    for k, v in l.items():
        if v == True:
            return (k, v)
        elif v == False:
            return (-k, v)
    return (None, None)

In [13]:
one_pure_symbol([A,B,C], [{A},{-B,-A},{C},{C,-B}])

(B, False)

In [14]:
one_pure_symbol([A,B,C], [{-A},{-B,A},{C},{-C,B}])

(None, None)

In [15]:
def common_pure_symbol(symbols, clauses):
    """ Return the most common pure symbol and its sign.  
    If no pure symbol found, return (None, None)
    """
    flat = [s for c in clauses for s in c if s in symbols]
    l = find_pure_symbols(symbols, clauses)
    freq = {}
    for k, v in l.items():
        if v == None:
            continue
        else:
            freq[k] = flat.count(k)
    if not freq:
        return (None, None)
    result = max(freq, key=freq.get)
    if l[result] == True:
        return (result, l[result])
    else:
        return (-result, l[result])

In [16]:
common_pure_symbol([A,B,C],[{A},{C},{C,-B},{C}])

(C, True)

In [17]:
common_pure_symbol([A,B,C], [{A},{-C,-B},{C,B,-A}])

(None, None)

In [18]:
def true_unit_clause(literal):
    """ Output the truth value of the literal 
    for the unit clause to be true. In this case,
    unit clause only contains one literal whose sign
    is either positive or negative. 
    """
    if literal.sign == True:
        return (literal, True)
    elif literal.sign == False:
        return (-literal, False)

In [19]:
true_unit_clause(C)

(C, True)

In [20]:
true_unit_clause(-C)

(C, False)

In [21]:
def unit_clause(clause, model):
    """ Check if a clause could be a unit clause. 
    Return the key symbol with the correct truth 
    value so that the clause is true.   
    """
    undefined = 0
    for symbol in clause:
        s, t = true_unit_clause(symbol)
        if s in model:
            #This symbol is assigned False by the model, continue to the next symbol
            if model[s] != t:
                continue
            #This symbol is assigned True by the model
            else:
                return (None, None)
        else:
            undefined +=1
        if undefined > 1:
            return (None, None)
        P, value = s, t 
    return (P, value)

In [22]:
unit_clause({A,C,D}, {A:True})

(None, None)

In [23]:
unit_clause({-A,-B}, {A:True})

(B, False)

In [24]:
def one_unit_clause(clauses, model):
    """ Given a list of clauses, return one of the 
    unit clauses. 
    """
    for c in clauses:
        s, v = unit_clause (c, model)
        if s:
            return (s, v)
    return (None, None)

In [25]:
one_unit_clause([{A,B,C},{B,-C},{-A,-B}],{A:True})

(B, False)

In [26]:
def removed(s,symbols):
    """ Return the list of remaining symbols after
    processing a symbol. 
    """
    return [x for x in symbols if x!= s]

In [27]:
removed(C,[A,B,C])

[A, B]

In [28]:
def added(model, s, value):
    """ Add a symbol into our model 
    after we have known its truth value  
    """
    mod = model.copy()
    mod[s] = value
    return(mod)

In [29]:
added({}, A, False)

{A: False}

In [30]:
def DPLL(clauses, symbols, model ={}):
    unknown = []
    #Loop through all the clauses and how they are evaluated by
    #the given model. 
    for c in clauses:
        val = is_true(c, model)
        if val is False:
            #When one clause is False, KB is False because of conjunctions.
            return False  
        if val is not True:
            #If the truth value of a clause is not decided from the KB, 
            #we append it to unknown. 
            unknown.append(c) 
    if not unknown:
        #After knowing the truth value all the symbols, we return the model
        return model
    
    #Pure Symbol Heuristics
    #P, value = find_pure_symbol(symbols, unknown)
    P, value = common_pure_symbol(symbols, unknown)
    if P:
        return DPLL(clauses, removed(P, symbols), 
                    added(model, P, value))

    # Unit Clause Heuristics 
    P, value = one_unit_clause(clauses, model)
    if P:
        return DPLL(clauses, removed(P, symbols),
                    added(model, P, value))
    
    # Degree Heuristics
    P, symbols = most_common(symbols, clauses)
    
    return (DPLL(clauses,symbols,added(model, P, True)) or
            DPLL(clauses,symbols,added(model, P, False)))

In [31]:
from collections import OrderedDict

def DPLL_satisfiable(KB):
    symbols = []
    for clause in KB:
        for symbol in clause:
            if symbol.sign == True:
                symbols.append(symbol)
            else:
                symbols.append(-symbol)
    symbols = list(set(symbols))        
    symbols = sorted(symbols, key=lambda x: x.name)
    model = DPLL(KB, symbols)
    if model == False:
        return (False, {})
    else:
        result = {}
        for s in symbols: 
            if s in model:
                result[s] = str(model[s]).lower()
            else:
                result[s] = "free"
        result = OrderedDict(sorted(result.items(), 
                                key=lambda i: i[0].name))
        return(True, result)

In [32]:
KB = [{-A,B,E},{-B, A},{-E, A},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]

In [33]:
DPLL_satisfiable(KB)

(True,
 OrderedDict([(A, 'false'),
              (B, 'false'),
              (C, 'free'),
              (D, 'true'),
              (E, 'false'),
              (F, 'free')]))